In [1]:
library(tidyverse)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.6
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



このページ https://github.com/tidyverse/dplyr/issues/4544 のデータを例にQで始まる列の和を求めるコードをいくつか紹介する。

In [2]:
QIMU_raw <- data.frame(
  ID = c(1L, 2L, 3L, 4L, 5L, 6L),
  Age = c(25L, 20L, 23L, 29L, 24L, 27L),
  Q1 = c(-0.49, -0.08, -2.01, 1, 0.27, -0.45),
  Q2 = c(0.4, 0.71, -0.77, 0.88, -1.92, -0.48),
  Q3 = c(0.53, 0.71, -1.02, -0.09, 0.17, -0.31),
  Q4 = c(-0.47, -0.51, -2.27, -0.26, -1.35, -1.45),
  Q5 = c(-0.44, 0.64, -1.05, 0.8, 0.83, 0.01),
  Q6 = c(-1.02, -0.02, -1.57, 0.07, -0.52, -0.9),
  Sex = as.factor(c("F", "F", "F", "F", "M", "M"))
)
QIMU_raw

ID,Age,Q1,Q2,Q3,Q4,Q5,Q6,Sex
<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,25,-0.49,0.40,0.53,-0.47,-0.44,-1.02,F
2,20,-0.08,0.71,0.71,-0.51,0.64,-0.02,F
3,23,-2.01,-0.77,-1.02,-2.27,-1.05,-1.57,F
4,29,1.00,0.88,-0.09,-0.26,0.80,0.07,F
5,24,0.27,-1.92,0.17,-1.35,0.83,-0.52,M
6,27,-0.45,-0.48,-0.31,-1.45,0.01,-0.90,M


このデータに対して、Q1からQ6までの列の和を求めて、Q_totalという列を作るのを目標とする！

Q1からQ6までしか列がなければ、apply()で簡単に書ける。第２引数の1で行方向の操作であることを指定している。

In [3]:
QIMU_raw %>% 
  select(Q1:Q6) %>%
  mutate(Q_total = apply(., 1, sum))

Q1,Q2,Q3,Q4,Q5,Q6,Q_total
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
-0.49,0.40,0.53,-0.47,-0.44,-1.02,-1.49
-0.08,0.71,0.71,-0.51,0.64,-0.02,1.45
-2.01,-0.77,-1.02,-2.27,-1.05,-1.57,-8.69
1.00,0.88,-0.09,-0.26,0.80,0.07,2.40
0.27,-1.92,0.17,-1.35,0.83,-0.52,-2.52
-0.45,-0.48,-0.31,-1.45,0.01,-0.90,-3.58


しかし、Qで始まる列以外も残したいときはどうするか？

In [4]:
QIMU_raw %>% 
  mutate(Q_total = apply(select(., Q1:Q6), 1, sum))

ID,Age,Q1,Q2,Q3,Q4,Q5,Q6,Sex,Q_total
<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>
1,25,-0.49,0.40,0.53,-0.47,-0.44,-1.02,F,-1.49
2,20,-0.08,0.71,0.71,-0.51,0.64,-0.02,F,1.45
3,23,-2.01,-0.77,-1.02,-2.27,-1.05,-1.57,F,-8.69
4,29,1.00,0.88,-0.09,-0.26,0.80,0.07,F,2.40
5,24,0.27,-1.92,0.17,-1.35,0.83,-0.52,M,-2.52
6,27,-0.45,-0.48,-0.31,-1.45,0.01,-0.90,M,-3.58


select()の列選択をstarts_with()などにするとより応用が広がる

In [5]:
QIMU_raw %>% 
  mutate(Q_total = apply(select(., starts_with("Q")), 1, sum))

ID,Age,Q1,Q2,Q3,Q4,Q5,Q6,Sex,Q_total
<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>
1,25,-0.49,0.40,0.53,-0.47,-0.44,-1.02,F,-1.49
2,20,-0.08,0.71,0.71,-0.51,0.64,-0.02,F,1.45
3,23,-2.01,-0.77,-1.02,-2.27,-1.05,-1.57,F,-8.69
4,29,1.00,0.88,-0.09,-0.26,0.80,0.07,F,2.40
5,24,0.27,-1.92,0.17,-1.35,0.83,-0.52,M,-2.52
6,27,-0.45,-0.48,-0.31,-1.45,0.01,-0.90,M,-3.58


apply()とsum()の組み合わせの代わりに、rowSums()を使うこともできる

In [6]:
QIMU_raw %>% 
  select(Q1:Q6) %>%  
  mutate(Q_total = rowSums(.))

Q1,Q2,Q3,Q4,Q5,Q6,Q_total
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
-0.49,0.40,0.53,-0.47,-0.44,-1.02,-1.49
-0.08,0.71,0.71,-0.51,0.64,-0.02,1.45
-2.01,-0.77,-1.02,-2.27,-1.05,-1.57,-8.69
1.00,0.88,-0.09,-0.26,0.80,0.07,2.40
0.27,-1.92,0.17,-1.35,0.83,-0.52,-2.52
-0.45,-0.48,-0.31,-1.45,0.01,-0.90,-3.58


Qで始まる以外の列も残したまま同じことを実現するには？

In [7]:
QIMU_raw %>% 
  mutate(Q_total = rowSums(select(., Q1:Q6)))

ID,Age,Q1,Q2,Q3,Q4,Q5,Q6,Sex,Q_total
<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>
1,25,-0.49,0.40,0.53,-0.47,-0.44,-1.02,F,-1.49
2,20,-0.08,0.71,0.71,-0.51,0.64,-0.02,F,1.45
3,23,-2.01,-0.77,-1.02,-2.27,-1.05,-1.57,F,-8.69
4,29,1.00,0.88,-0.09,-0.26,0.80,0.07,F,2.40
5,24,0.27,-1.92,0.17,-1.35,0.83,-0.52,M,-2.52
6,27,-0.45,-0.48,-0.31,-1.45,0.01,-0.90,M,-3.58


In [8]:
QIMU_raw %>% 
  mutate(Q_total = rowSums(select(., starts_with("Q"))))

ID,Age,Q1,Q2,Q3,Q4,Q5,Q6,Sex,Q_total
<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>
1,25,-0.49,0.40,0.53,-0.47,-0.44,-1.02,F,-1.49
2,20,-0.08,0.71,0.71,-0.51,0.64,-0.02,F,1.45
3,23,-2.01,-0.77,-1.02,-2.27,-1.05,-1.57,F,-8.69
4,29,1.00,0.88,-0.09,-0.26,0.80,0.07,F,2.40
5,24,0.27,-1.92,0.17,-1.35,0.83,-0.52,M,-2.52
6,27,-0.45,-0.48,-0.31,-1.45,0.01,-0.90,M,-3.58


さらに、よりモダンなtidyverse-likeな書き方も見てみよう

複数の列に対して操作を実現するacross()を使ってみよう

In [9]:
QIMU_raw %>%
  mutate(Q_total = rowSums(across(Q1:Q6)))

ID,Age,Q1,Q2,Q3,Q4,Q5,Q6,Sex,Q_total
<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>
1,25,-0.49,0.40,0.53,-0.47,-0.44,-1.02,F,-1.49
2,20,-0.08,0.71,0.71,-0.51,0.64,-0.02,F,1.45
3,23,-2.01,-0.77,-1.02,-2.27,-1.05,-1.57,F,-8.69
4,29,1.00,0.88,-0.09,-0.26,0.80,0.07,F,2.40
5,24,0.27,-1.92,0.17,-1.35,0.83,-0.52,M,-2.52
6,27,-0.45,-0.48,-0.31,-1.45,0.01,-0.90,M,-3.58


In [10]:
QIMU_raw %>%
  mutate(Q_total = rowSums(across(starts_with("Q"))))

ID,Age,Q1,Q2,Q3,Q4,Q5,Q6,Sex,Q_total
<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>
1,25,-0.49,0.40,0.53,-0.47,-0.44,-1.02,F,-1.49
2,20,-0.08,0.71,0.71,-0.51,0.64,-0.02,F,1.45
3,23,-2.01,-0.77,-1.02,-2.27,-1.05,-1.57,F,-8.69
4,29,1.00,0.88,-0.09,-0.26,0.80,0.07,F,2.40
5,24,0.27,-1.92,0.17,-1.35,0.83,-0.52,M,-2.52
6,27,-0.45,-0.48,-0.31,-1.45,0.01,-0.90,M,-3.58


across()の詳しい説明は https://dplyr.tidyverse.org/reference/across.html （ただし、詳しすぎて混乱するかも）

また、行方向に操作することを明示的に行うrowwise()と組み合わせることもできる。
さらに、c_across()で列の範囲を指定できる。

In [11]:
QIMU_raw %>% 
  rowwise() %>%
  mutate(Q_total = sum(c_across(Q1:Q6)))

ID,Age,Q1,Q2,Q3,Q4,Q5,Q6,Sex,Q_total
<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>
1,25,-0.49,0.40,0.53,-0.47,-0.44,-1.02,F,-1.49
2,20,-0.08,0.71,0.71,-0.51,0.64,-0.02,F,1.45
3,23,-2.01,-0.77,-1.02,-2.27,-1.05,-1.57,F,-8.69
4,29,1.00,0.88,-0.09,-0.26,0.80,0.07,F,2.40
5,24,0.27,-1.92,0.17,-1.35,0.83,-0.52,M,-2.52
6,27,-0.45,-0.48,-0.31,-1.45,0.01,-0.90,M,-3.58


この場合は、rowSums()ではなく、sum()になっていることに注意しよう。なぜなら、rowwise()ですでに行方向に操作をすることは指定済みのため。

rowwise()の詳しい説明はここ https://dplyr.tidyverse.org/articles/rowwise.html

さらに発展編として、purrrパッケージのpmap_***()を使って実現することもできる。

In [12]:
QIMU_raw %>% 
  mutate(Q_total = pmap_dbl(select(., starts_with("Q")), sum))

ID,Age,Q1,Q2,Q3,Q4,Q5,Q6,Sex,Q_total
<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>
1,25,-0.49,0.40,0.53,-0.47,-0.44,-1.02,F,-1.49
2,20,-0.08,0.71,0.71,-0.51,0.64,-0.02,F,1.45
3,23,-2.01,-0.77,-1.02,-2.27,-1.05,-1.57,F,-8.69
4,29,1.00,0.88,-0.09,-0.26,0.80,0.07,F,2.40
5,24,0.27,-1.92,0.17,-1.35,0.83,-0.52,M,-2.52
6,27,-0.45,-0.48,-0.31,-1.45,0.01,-0.90,M,-3.58


新しく作成されるQ_total列がdblの型のため、ここで用いるpmap系の関数もpmap_dbl()となる。  
しかし、難しすぎるので覚えなくてよい

最後に、  
不要な列を削除してから複数の列の和を求めて、もともとあった列を付け足すのでも、もちろんよい。   

上に列挙した書き方のバリエーションはリファレンスを見ながら試行錯誤で編み出したため、  
正直、リファレンスを見ずに適切な関数と引数の与え方を書ける自信はまったくない。

一番わかりやすい、読みやすい、間違いの少ない書き方がひとつできればそれでよい。  
シンプルな自分に合った書き方が見つかれば、それを使っていければよい。
